In [ ]:
#importing pyspark
import pyspark

In [ ]:
#importing sparksession
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("airline").getOrCreate()

In [ ]:
spark.stop()

# ################################

In [ ]:
#To create dataframe form External datasets
df = spark.read.option("header", "true").csv("airlines1.csv")

In [ ]:
# for few columns
df.columns[:5]

In [ ]:
# for all columns
df.columns

# Create multiple dataframes as per need

In [ ]:
Flight_Details = df.select("_c0","Year", "Month", "DayofMonth" , "FlightDate","Tail_Number", "Flight_Number_Reporting_Airline" )

In [ ]:
#count total no of flights in each years 
Flight_Details.select('Year').groupBy('Year').count().show()

In [ ]:
#count total no of flights in each months
Flight_Details.select('Month').groupBy('Month').count().show()

In [ ]:
#count total no of cancelled flights in each years

In [ ]:
df.select('Year').filter("Cancelled = 1").groupBy('Year').count().show()

In [ ]:
#count total no of cancelled flights in each months

In [ ]:
df.select('Month').filter("Cancelled = 1").groupBy('Month').count().show()

In [ ]:
#count total no of flights By departure airports

In [ ]:
df.select('Origin').groupBy('Origin').count().show()

In [ ]:
#calculate years wise on time departure flights

In [ ]:
df.select('Year').filter("DepDelay <= 0").groupBy('Year').count().show()

In [ ]:
#calculate years wise departure delay flights
df.select('Year').filter("DepDelay > 0").groupBy('Year').count().show()

In [ ]:
#calculate years wise permormance of flight like on time departure, on time arrival

In [ ]:
df.select('Year').filter((df.DepDelay <= 0) & (df.ArrDelay <= 0)).groupBy('Year').count().show()

In [ ]:
Flight_Origin= df.select("_c0","OriginAirportID", "OriginAirportSeqID", "OriginCityMarketID","Origin", "OriginCityName","OriginState", "OriginStateName")

In [ ]:
Flight_Destination = df.select("_c0","DestAirportID", "DestAirportSeqID", "Dest", "DestCityName","DestState","DestStateName")

In [ ]:
Flight_arr_dep = df.select("_c0","CRSArrTime","ArrTime", "CRSDepTime","DepTime")

In [ ]:
DelayDF= df.select("_c0","CarrierDelay","WeatherDelay", "NASDelay","SecurityDelay","LateAircraftDelay")

In [ ]:
Flight_cancel = df.select("_c0","Cancelled","CancellationCode","Diverted","Flights")

In [ ]:
#modify column name
# to modify multiple column name
#Flight_cancel.withColumnRenamed("_c0","ID").withColumnRenamed("CancellationCode","Code")

In [ ]:
Flight_cancel = Flight_cancel.withColumnRenamed("_c0","ID")

In [ ]:
Flight_cancel.show()

# Data Cleaning
# handling null values

In [ ]:
Flight_cancel.show()

In [ ]:
Flight_cancel.na.drop().show()

In [ ]:
Flight_cancel.na.drop(how='any').show()

In [ ]:
Flight_cancel.na.fill("yes").show()

# apply pyspark windows function

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [ ]:
ndf  = Window.partitionBy("OriginState").orderBy("OriginAirportID")
Flight_Origin.withColumn("row_number",row_number().over(ndf)).show()


In [ ]:

# Use rank function
from pyspark.sql.functions import rank
Flight_Origin.withColumn("rank",rank().over(ndf)).show()


In [ ]:
# sorting dataframe by using sort
Flight_Origin.sort("Origin","OriginState").show()


In [ ]:
# sorting datafranme by orderby
Flight_Origin.orderBy("Origin","OriginState").show()


In [ ]:
# sort by ascending
#Flight_Origin.sort(Flight_Origin.Origin.asc(),df.OriginState.asc()).show()
#Flight_Origin.orderBy(col("Origin").asc(),col("OriginState").asc()).show()


In [ ]:
# sort by desciending 
#Flight_Origin.sort(Flight_Origin.Origin.desc(),df.OriginState.desc()).show()
#Flight_Origin.orderBy(col("Origin").desc(),col("OriginState").desc()).show()


In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [ ]:
Flight_Destination.show()

# how to use UDF in pyspark DataFrame

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
from pyspark.sql.functions import col

In [ ]:
#from pyspark.sql.types import StructType, IntegerType, StringType
# as per datatype StructType, IntegerType, StringType in columns we need to import these library 
#otherwise bydefaut it wil take StringType

In [ ]:
#create function in python

In [ ]:
def destination(str):
    resStr=""
    arr = str.split(",")
    for x in arr:
       resStr= arr[0]
    return resStr 


In [ ]:
#Converting python function to UDF
destinationUDF = udf(lambda z: destination(z))

In [ ]:
df.select(col("Dest"),destinationUDF(col("DestCityName")).alias("DestCityName") ).show(truncate=False)

In [ ]:
Flight_Destination.show()